In [1]:
import numpy as np
import astropy.coordinates
from astropy import units as u

In [19]:
coord=astropy.coordinates.SkyCoord(ra=10.625*u.degree, dec=41.2*u.degree, pm_ra=2*u.mas/u.yr, pm_dec=2*u.mas/u.yr, frame='icrs')

ValueError: Unrecognized keyword argument(s) 'pm_ra'

 By default, most frame classes expect the longitudinal proper motion to include the cos(latitude) term, named `pm_ra_cosdec`. Did you mean to pass in this component?

In [14]:
print(coord.barycentrictrueecliptic)
print(coord.barycentrictrueecliptic.lon.degree)
deg=coord.barycentrictrueecliptic.lon.degree
print(2*deg)
#print(dir(coord))

<SkyCoord (BarycentricTrueEcliptic: equinox=J2000.000): (lon, lat) in deg
    (27.7602082, 33.30810693)>
27.76020820089444
55.52041640178888
